# Config

# Testing TensorRT endpoint

In [32]:
import requests




import base64
import numpy as np
import scipy.io  as io
import pandas as pd 


path = "/home/akinwilson/Code/pytorch/dataset/keywords/yes/yes.f5626af6_nohash_2.wav"
train_df_path = "/home/akinwilson/Code/pytorch/dataset/keywords/train.csv"
df = pd.read_csv(train_df_path).sample(n=100)

def encode_wav(path):
    _, wav = io.wavfile.read(path)
    wav
    s = base64.b64encode(wav)
    return s 


data_list = [{'base64str': encode_wav(path).decode('utf-8')} for path in df.wav_path]


def send_request(data):
    url = 'http://0.0.0.0:8080/api/v1/predict'
    x = requests.post(url, json = data)
    return x.json()


# x = send_request(data_list[0])
# status  = x['error']
# response = x['result']


import multiprocessing as mp
import time 
pool = mp.Pool(processes=40)
s = time.time()
results = pool.map(send_request, data_list)
f = time.time()
print(f"Sent {len(data_list)} queries in {f-s} seconds")
print(f"QPS: {(len(data_list) / (f-s) ):.2f}")
# print "\nTotal took " + str(millis() - start_time) + " ms\n"
# for result in results:
inf_times = [result['result']['inference_time'] for result in results] 
avg_inf = sum(inf_times) / len(inf_times)
print(f"Avg inference time: {avg_inf}")


Sent 100 queries in 4.238741397857666 seconds
QPS: 23.59
Avg inference time: 1.4751449227333069


In [ ]:
path = "/home/akinwilson/Code/pytorch/dataset/data/000a5b70-dfe9-4bf5-9199-85feb935bfdb.wav"

from scipy.io import wavfile
import base64 

samplerate, data = wavfile.read(path)


txt = base64.b64encode(data)

# with open("bas64.example", "wb") as file:
#     file.write(txt)

import numpy as np 

decoded_wav = base64.decodebytes(txt)
x = np.frombuffer(decoded_wav, dtype=np.int16)
x.shape


In [ ]:
import os 

print()

## Testing onnx runtime

 Issue with CUDA as exection provider. 

 see: 
 
    https://stackoverflow.com/questions/70014477/onnxruntime-not-using-cuda

In [ ]:
ort_session.get_providers()

In [ ]:
import onnx 

# path= "/home/akinwilson/Code/pytorch/output/model/DeepSpeech/2022_09_12_11.22.09_AM/export/DeepSpeech.onnx"

# p = "/home/akinwilson/Code/pytorch/output/model/DeepSpeech/2022_09_12_11.22.09_AM/inference/DeepSpeech.pt"

import onnxruntime
import torch 
import time 
import logging 
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

x = torch.randn(1, 1, 48000, device='cpu')
s = time.time()


In [ ]:
# import onnxruntime as ort
# ort.get_all_providers()
# ort.get_device()

In [ ]:
# import onnxruntime
# import torch 
# import time 
# onnxruntime.get_providers()

In [ ]:
# p = "/home/akinwilson/Code/pytorch/output/model/DeepSpeech/2022_09_12_11.22.09_AM/inference/DeepSpeech.pt"
# import torch 
# import time 
# # model = torch.load(p)
# s= time.time()
# model.forward(x)
# f = time.time()
# print(f"Serving python: {(f-s)* 1e3:.5f}ms")